In [ ]:
!pip install dlt
#SQL TO SQL

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 918.2/918.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 16.1 MB/s eta 0:00:00


In [ ]:
#Backfilling

In [ ]:
data = [
    {"name": "Vincent Crabbe", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Gregory Goyle", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Draco Malfoy", "designation": "student", "date_started": "1991-09-01T09:00:00Z"}
]

In [ ]:
#Slowly changing dimensions
import dlt

pipeline = dlt.pipeline(
    pipeline_name="hogwarts_pipeline",
    destination="duckdb",
    dataset_name="hogwarts",
)

load_info = pipeline.run(
    data,
    table_name="creatures",
    write_disposition={
        "disposition": "merge", # <--- specifies that existing data should be merged rather than replaced
        "strategy": "scd2" # <--- enables SCD2 tracking, which keeps historical records of changes
    }
)
print(load_info)



Pipeline hogwarts_pipeline load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset hogwarts
The duckdb destination used duckdb:////content/hogwarts_pipeline.duckdb location to store data
Load package 1745800778.3532386 is LOADED and contains no failed jobs


In [ ]:
pipeline.dataset(dataset_type="default").creatures.df()

,_dlt_valid_from,_dlt_valid_to,name,designation,date_started,_dlt_load_id,_dlt_id
0,2025-04-28 00:39:38.353239+00:00,NaT,Vincent Crabbe,student,1991-09-01 09:00:00+00:00,1745800778.3532386,P94fUt8jUkzx6A
1,2025-04-28 00:39:38.353239+00:00,NaT,Gregory Goyle,student,1991-09-01 09:00:00+00:00,1745800778.3532386,Wv4kISjkizZwkQ
2,2025-04-28 00:39:38.353239+00:00,NaT,Draco Malfoy,student,1991-09-01 09:00:00+00:00,1745800778.3532386,r7u0vNojh+gvEQ


In [ ]:
#two new columns where added above to track the changes in the data, data is considered not valid when it is updated. lets update the data
data = [
    {"name": "Vincent Crabbe", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Gregory Goyle", "designation": "student", "date_started": "1991-09-01T09:00:00Z"},
    {"name": "Draco Malfoy", "designation": "expelled", "date_started": "1991-09-01T09:00:00Z"}
]

In [ ]:
#running the pipeline again to update the data
load_info = pipeline.run(
    data,
    table_name="creatures",
    write_disposition={
        "disposition": "merge",
        "strategy": "scd2"
    }
)
print(load_info)

Pipeline hogwarts_pipeline load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset hogwarts
The duckdb destination used duckdb:////content/hogwarts_pipeline.duckdb location to store data
Load package 1745800987.5920975 is LOADED and contains no failed jobs


In [ ]:
#lets check to see if the updat to draco malfoy who was expelled is captured
pipeline.dataset(dataset_type="default").creatures.df()

,_dlt_valid_from,_dlt_valid_to,name,designation,date_started,_dlt_load_id,_dlt_id
0,2025-04-28 00:39:38.353239+00:00,NaT,Vincent Crabbe,student,1991-09-01 09:00:00+00:00,1745800778.3532386,P94fUt8jUkzx6A
1,2025-04-28 00:39:38.353239+00:00,NaT,Gregory Goyle,student,1991-09-01 09:00:00+00:00,1745800778.3532386,Wv4kISjkizZwkQ
2,2025-04-28 00:39:38.353239+00:00,2025-04-28 00:43:07.592098+00:00,Draco Malfoy,student,1991-09-01 09:00:00+00:00,1745800778.3532386,r7u0vNojh+gvEQ
3,2025-04-28 00:43:07.592098+00:00,NaT,Draco Malfoy,expelled,1991-09-01 09:00:00+00:00,1745800987.5920975,KCWs+o6jOW/aLQ


In [ ]:
#changing storage, sql to sql
pipeline = dlt.pipeline(
    pipeline_name='github_pipeline',
    destination='duckdb', # <--- to test pipeline locally
    dataset_name='events_dataset',
)

import os
from google.colab import userdata

os.environ["DESTINATION__BIGQUERY__CREDENTIALS"] = userdata.get('BIGQUERY_CREDENTIALS') #credentials for new destination

pipeline = dlt.pipeline(
    pipeline_name='github_pipeline',
    destination='bigquery', # <--- to run pipeline in production
    dataset_name='events_dataset',
)

In [ ]:
#changing the destination a files system
import dlt

pipeline = dlt.pipeline(
    pipeline_name='fs_pipeline',
    destination='filesystem', # <--- change destination to 'filesystem'
    dataset_name='fs_data',
)

load_info = pipeline.run(
    paginated_getter,
    loader_file_format="parquet", # <--- choose a file format: parquet, csv or jsonl
    write_disposition="append" # <--- filesystem destinaition doesn't support merge will fall back to `append`
)
print(pipeline.last_trace)

In [ ]:
#from local to cloud storage (to iceberg)
!pip install "dlt[pyiceberg]"

In [ ]:
pipeline = dlt.pipeline(
    pipeline_name='fs_iceberg_pipeline',
    destination='filesystem',
    dataset_name='fs_iceberg_data',
)

load_info = pipeline.run(
    paginated_getter,
    loader_file_format="parquet",
    write_disposition="append",
    table_format="iceberg",  # <--- choose a table format: delta or iceberg
)
print(pipeline.last_trace)

In [ ]:
#to s3
!pip install "dlt[s3]"

import os
from google.colab import userdata
import dlt

# another way to set your credentials for dlt pipelines, you can use ENVs as well
dlt.secrets["destination.filesystem.credentials.aws_access_key_id"] = userdata.get('AWS_ACCESS_KEY_ID')
dlt.secrets["destination.filesystem.credentials.aws_secret_access_key"] = userdata.get('AWS_SECRET_ACCESS_KEY')
dlt.secrets["destination.filesystem.credentials.region_name"]= "eu-central-1"

#specify bucket url
import os
dlt.secrets["destination.filesystem.bucket_url"]= "s3://gtm-demos/freecodecamp-demo/


#pipeline changes
pipeline = dlt.pipeline(
    pipeline_name='fs_s3_iceberg_pipeline',
    destination='filesystem', # <--- change destination to 'filesystem'
    dataset_name='fs_s3_iceberg_data',
)

#run the pipeline
load_info = pipeline.run(
    paginated_getter,
    loader_file_format="parquet", # <--- choose a file format: parquet, csv or jsonl
    write_disposition="append", # <--- filesystem destinaition doesn't support merge will fall back to `append`
    # table_format="iceberg",
)
print(pipeline.last_trace)

pipeline.dataset(dataset_type="default").events.df().head()

In [ ]:
#deploying a pipeline to GitHub actions